# RAG (Retrieval-Augmented Generation)

O RAG é uma técnica que combina recuperação de informações (retrieval) com geração de linguagem natural (generation) para melhorar a qualidade e a precisão das respostas de modelos como o ChatGPT.

In [ ]:
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv
import tiktoken
import os

load_dotenv()

model = "gpt-4o-mini"
llm = OpenAI(model=model, temperature=0)
tokenizer = tiktoken.encoding_for_model(model)

collection_name = "contracts_info"

question = "O que fala a cláusula 7 do contrato de locação de maquinário?"
# question = "O que fala a cláusula 7?" # Usuário não especificou o contrato

contents = []
for file in os.listdir("docs"):
    if file.endswith(".md"):
        with open(f"docs/{file}", "r") as f:
            content = f.read()
            contents.append({"filename": file, "content": content})

## Acessando os dados sem RAG

In [2]:
from llama_index.core.prompts import ChatMessage

non_rag_prompt = f"""
    Você é um especialista em leitura e extração de dados em contratos. Seu papel é ler os documentos fornecidos e responder perguntas sobre eles.

    Documentos:
    {contents}
"""

messages = [
    ChatMessage(role="system", content=non_rag_prompt),
    ChatMessage(role="user", content=question),
]

response = llm.chat(messages)
print(f"Tokens de entrada: {len(tokenizer.encode(non_rag_prompt + question))}")
print(f"Tokens de saída: {len(tokenizer.encode(str(response)))}")
print(
    f"Total de tokens: {len(tokenizer.encode(non_rag_prompt + question + str(response)))}"
)
print(f"Resposta: {response}")

Tokens de entrada: 7756
Tokens de saída: 128
Total de tokens: 7884
Resposta: assistant: A Cláusula 7 do contrato de locação de maquinário trata dos deveres da LOCATÁRIA. Ela se compromete a:

a) Confiar à LOCADORA o direito de fiscalização do maquinário arrendado;
b) Defender a posse e a propriedade das referidas máquinas;
c) Manter sempre um mínimo de três funcionários treinados pela LOCADORA, para realização da execução dos serviços específicos do maquinário;
d) Realizar o pagamento de quaisquer defeitos ou danos causados ao maquinário, bem como qualquer uma das máquinas pertencentes a este conjunto.


## Criação de uma base vetorial para o RAG usando ChromaDB

In [ ]:
import json
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.schema import TextNode
from chromadb import PersistentClient
from llama_index.vector_stores.chroma import ChromaVectorStore

os.makedirs("database", exist_ok=True)
chroma_client = PersistentClient(path="database")
chroma_collection = chroma_client.get_or_create_collection(name=collection_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

input_tokens = 0
output_tokens = 0

for data in contents:
    filename = data["filename"]
    content = data["content"]

    prompt = f"""
        Você é um especialista em leitura e extração de dados em contratos. Seu papel é ler o documento fornecido e separar blocos de texto com base em sua estrutura contratual.

        Regras
            - Não insira mais de uma cláusula dentro de um mesmo bloco;
            - Nos metadados do bloco, sempre informe o filename ({filename});
            - Se o conteúdo do bloco estiver inserido em uma seção ou cláusula identificável, inclua o campo "section" e/ou "clausula" nos metadados;
            - Cada bloco de texto não deve ultrapassar 5.000 caracteres;
            - A resposta deve ser uma lista JSON contendo os campos "metadata" e "content" para cada item;
            - Não adicione comentários, interpretações ou dados externos;
            - Não modifique ou omita nenhuma informação ou caractere do texto original.

        Exemplo de saída:
        [
            {{
                "metadata": {{
                "filename": "teste.docx",
                "section": "IDENTIFICAÇÃO DAS PARTES CONTRATANTES"
                }},
                "content": "**CONTRATO DE TESTE**\\nIDENTIFICAÇÃO DAS PARTES CONTRATANTES\\nCONTRATANTE: (Nome da Contratante), ..."
            }},
            {{
                "metadata": {{
                "filename": "teste.docx",
                "section": "DO OBJETO DO CONTRATO",
                "clausula": "1ª"
                }},
                "content": "Cláusula 1ª. O presente contrato tem como OBJETO, ..."
            }},
            {{
                "metadata": {{
                "filename": "teste.docx",
                }},
                "content": "Por estarem assim justos e contratados, ..."
            }}
        ]

        Documento:
        {content}
    """

    i = 0
    while i < 3:
        try:
            response = llm.complete(prompt).text
            break
        except Exception as e:
            i += 1

    if "```json" in response:
        response = response.split("```json")[1].split("```")[0].strip()
    elif "```" in response:
        response = response.split("```")[1].split("```")[0].strip()

    chunks_list = json.loads(response)

    input_tokens += len(tokenizer.encode(prompt))
    output_tokens += len(tokenizer.encode(response))

    nodes = []
    for chunk in chunks_list:
        metadata = chunk.get("metadata", {})
        content = chunk.get("content", "")

        node = TextNode(text=content, metadata=metadata)
        nodes.append(node)

    VectorStoreIndex(nodes, storage_context=storage_context)

print(f"Tokens de entrada: {input_tokens}")
print(f"Tokens de saída: {output_tokens}")
print(f"Total de tokens: {input_tokens + output_tokens}")

Tokens de entrada: 8476
Tokens de saída: 9658
Total de tokens: 18134


## RAG com ChromaDB

### Acessando os dados com RAG

In [ ]:
from llama_index.core.indices.vector_store import VectorIndexRetriever
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import ResponseMode

chroma_client = PersistentClient(path="database")
chroma_collection = chroma_client.get_or_create_collection(name=collection_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

retriever = VectorIndexRetriever(index=index)

retrieved_nodes = retriever.retrieve(question)

context_content = ""
for node in retrieved_nodes:
    context_content += node.text + "\n"

query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    llm=llm,
    response_mode=ResponseMode("compact"),
)

response = query_engine.query(question)

print(f"Tokens de entrada: {len(tokenizer.encode(question + context_content))}")
print(f"Tokens de saída: {len(tokenizer.encode(str(response)))}")
print(
    f"Total de tokens: {len(tokenizer.encode(question + context_content + str(response)))}"
)
print(f"Número de chunks recuperados: {len(retrieved_nodes)}")
print(f"Resposta: {response}")

Tokens de entrada: 242
Tokens de saída: 84
Total de tokens: 326
Número de chunks recuperados: 2
Resposta: A cláusula 7 do contrato de locação de maquinário estabelece os deveres da locatária. A locatária deve comunicar imediatamente qualquer ameaça ao maquinário, permitir a fiscalização pela locadora, defender a posse e propriedade das máquinas, manter pelo menos três funcionários treinados pela locadora para operar o maquinário, e pagar por quaisquer defeitos ou danos causados ao maquinário.


### Filtrando os chunks usando seus metadados

In [ ]:
from llama_index.core.indices.vector_store import VectorIndexRetriever
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters

chroma_client = PersistentClient(path="database")
chroma_collection = chroma_client.get_or_create_collection(name=collection_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

filters = MetadataFilters(
    filters=[
        MetadataFilter(key="filename", value="docs/Terraplanagem.md"),
    ]
)

retriever = VectorIndexRetriever(index=index, filters=filters)

retrieved_nodes = retriever.retrieve(question)

context_content = ""
for node in retrieved_nodes:
    context_content += node.text + "\n"

query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    llm=llm,
    response_mode=ResponseMode("compact"),
)

response = query_engine.query(question)

print(f"Tokens de entrada: {len(tokenizer.encode(question + context_content))}")
print(f"Tokens de saída: {len(tokenizer.encode(str(response)))}")
print(
    f"Total de tokens: {len(tokenizer.encode(question + context_content + str(response)))}"
)
print(f"Número de chunks recuperados: {len(retrieved_nodes)}")
print(f"Resposta: {response}")

Tokens de entrada: 18
Tokens de saída: 2
Total de tokens: 20
Número de chunks recuperados: 0
Resposta: Empty Response
